# Making bets on NBA games using Bayesian Neural Networks
The goal of this notebook is to explore the use of BNNs in predicting the outcome of NBA games. While using MLPs as seen in ```mlp_betting.ipynb``` may be computationally more efficient, personal testing has shown that tradional neural networks are overconfident in predictions making them unsuitable for betting. By learning the distributions of weights, BNNs can hopefully provide a better estimate on the outcome of games for use in betting.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,GridSearchCV,cross_validate,cross_val_predict
from sklearn.metrics import classification_report,confusion_matrix,make_scorer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from util.client import Nba_Season
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pyro
import pyro.distributions as dist
from pyro.infer import SVI, Trace_ELBO
from pyro.nn import PyroModule, PyroSample
import torch
import torch.nn as nn
from torch.nn import Softmax
from pyro.infer import MCMC, NUTS
from pyro.infer import Predictive

c:\Users\andre\OneDrive\Desktop\CS\betting_bot\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Define simple BNN using Pyro containing 1 hidden layer

For this implementation, we will be using the [Pyro Probablistic Programming language](https://github.com/pyro-ppl/pyro), loosely following a [tutorial](https://colab.research.google.com/drive/1NQNMdKaE9RncuWgO_vM2k3qywV76Byfh) from the University of Amsterdam

Currently, the model will only be predicting the outcomes of games (home win or away win) and compare outcomes to moneyline odds from [vegas insider](https://www.vegasinsider.com/nba/odds/las-vegas/). Because of this, the model will be learning a categorical output, 0 indicating a home win and 1 indicating away win. The model will sample each layers weights and biases from a normal distribution while the prediction will be sampled from a categorical distribution based on the output of the layers.

In [2]:
class BNN(PyroModule):
    def __init__(self, in_dim=1, out_dim=1, hid_dim=5, prior_scale=1.):
        super().__init__()

        self.activation = nn.ReLU()
        self.layer1 = PyroModule[nn.Linear](in_dim, hid_dim)
        self.layer2 = PyroModule[nn.Linear](hid_dim, out_dim)

        # Set layer parameters as random variables
        self.layer1.weight = PyroSample(dist.Normal(0., prior_scale).expand([in_dim, hid_dim]).to_event(2))
        self.layer1.bias = PyroSample(dist.Normal(0., prior_scale).expand([hid_dim,]).to_event(1))
        self.layer2.weight = PyroSample(dist.Normal(0., prior_scale).expand([hid_dim, out_dim]).to_event(2))
        self.layer2.bias = PyroSample(dist.Normal(0., prior_scale).expand([out_dim,]).to_event(1)) # output bias term

    def forward(self, x, y=None):
        z1 = self.activation(x @ self.layer1.weight + self.layer1.bias) # pass thru first layer
        z2 = self.activation(z1 @ self.layer2.weight + self.layer2.bias) # pass thru output layer

        #, x.shape[0]
        with pyro.plate("data"):
            y_hat = Softmax(dim=1)(z2) # scale output via softmax for setting categorical priors
            # y_hat = Softmax(dim=0)(x)
            obs = pyro.sample("obs", dist.Categorical(y_hat), obs=y)

        return z2

In [7]:
# load old samples and features
features = np.genfromtxt('../NBA/samps_feats/2015-2023_nba_features_norm_inj.csv',delimiter=',')
samples = np.genfromtxt('../NBA/samps_feats/2015-2023_nba_samples_inj.csv',delimiter=',')
samples_1d = [0 if j[0] == 0 else 1 for j in samples]
feat_train, feat_test, samp_train, samp_test = train_test_split(features, samples, test_size=0.25, random_state=1)
x_train = torch.FloatTensor(feat_train)
x_test = torch.FloatTensor(feat_test)
y_train = torch.FloatTensor(samp_train)
y_test = torch.FloatTensor(samp_test)

In [66]:
from pyro.infer import MCMC, NUTS

model = BNN(in_dim=16,hid_dim=16,out_dim=2)

pyro.set_rng_seed(1)

# Define Hamiltonian Monte Carlo (HMC) kernel
# NUTS = "No-U-Turn Sampler" (https://arxiv.org/abs/1111.4246), gives HMC an adaptive step size
nuts_kernel = NUTS(model, jit_compile=False)  # jit_compile=True is faster but requires PyTorch 1.6+

# Define MCMC sampler, get 50 posterior samples
mcmc = MCMC(nuts_kernel, num_samples=50)

mcmc.run(x_train, y_train.T)

Warmup:   0%|          | 0/100 [00:00, ?it/s]

Sample: 100%|██████████| 100/100 [07:31,  4.51s/it, step size=1.17e-02, acc. prob=0.450]


In [81]:
from pyro.infer import Predictive

# get predictions based on posteriors
predictive = Predictive(model=model, posterior_samples=mcmc.get_samples(), return_sites=['obs','_RETURN'])

train_preds = predictive(x_train)['obs'].T.float().mean(axis=1) # each x in training produces 50 predictions (0 or 1), take average
test_preds = predictive(x_test)['obs'].T.float().mean(axis=1)

adj_train_preds = [0 if p < 0.5 else 1 for p in train_preds]
adj_test_preds = [0 if p < 0.5 else 1 for p in test_preds]
y_train_1d = [0 if j[0] == 0 else 1 for j in y_train] # [0,1] -> [away,home] -> 0 indicates home win, 1 indicates away
y_test_1d = [0 if j[0] == 0 else 1 for j in y_test]

print('---TRAINING SET---')
print('TN, FP, FN, TP')
print(confusion_matrix(y_train_1d,adj_train_preds).ravel())
print(classification_report(y_train_1d,adj_train_preds))
print('---TEST SET---')
print('TN, FP, FN, TP')
print(confusion_matrix(y_test_1d,adj_test_preds).ravel())
print(classification_report(y_test_1d,adj_test_preds))

---TRAINING SET---
TN, FP, FN, TP
[1969 2431 1894 2343]
              precision    recall  f1-score   support

           0       0.51      0.45      0.48      4400
           1       0.49      0.55      0.52      4237

    accuracy                           0.50      8637
   macro avg       0.50      0.50      0.50      8637
weighted avg       0.50      0.50      0.50      8637

---TEST SET---
TN, FP, FN, TP
[678 813 596 793]
              precision    recall  f1-score   support

           0       0.53      0.45      0.49      1491
           1       0.49      0.57      0.53      1389

    accuracy                           0.51      2880
   macro avg       0.51      0.51      0.51      2880
weighted avg       0.51      0.51      0.51      2880



### Define kelly critereon to take in average prediction score and make bets
For placing bets, the predictions from the BNN will be used on a modified version of the [kelly critereon](https://en.wikipedia.org/wiki/Kelly_criterion) betting strategy, defined in the function ```kelly``` wrapped by ```BNN_kelly```.

In [3]:
def kelly(home_pred,away_pred,home_line,away_line,max_bet=100):
    '''
    Applies kelly critereon based on features and moneyline data
    home_pred: Prediction from MLP for home team
    away_pred: Prediction from MLP for away team
    home_line: Moneyline for home team
    away_line: Moneyline for away team
    '''
    bet_amount = 0
    to_win = 0

    log_home = home_pred - home_pred * away_pred / (home_pred + away_pred - (2*home_pred*away_pred))
    log_away = away_pred - home_pred * away_pred / (home_pred + away_pred - (2*home_pred*away_pred))

    # calculate ratio and implied for home
    home_line_adj = home_line
    away_line_adj = away_line
    if home_line < 0:
        home_line_adj *= -1
        home_line_adj /= 100
        home_ratio = 1/(home_line_adj)
        implied_home = home_line_adj/(1+home_line_adj)
    else:
        home_line_adj /= 100
        home_ratio = home_line_adj
        implied_home = 1/(home_line+1)

    # calculate ratio and implied for away
    if away_line < 0:
        away_line_adj *= -1
        away_line_adj /= 100
        away_ratio = 1/(away_line_adj)
        implied_away = away_line_adj/(1+away_line_adj)
    else:
        away_line_adj /= 100
        away_ratio = away_line_adj
        implied_away = 1/(away_line_adj+1)
    
    diff_home = log_home - implied_home
    diff_away = log_away - implied_away

    kelly_home = log_home - (log_away/home_ratio)
    kelly_away = log_away - (log_home/away_ratio)

    prob = 0

    # make bets, negative if away team bet
    if diff_home > diff_away and diff_home > 0.05:
        bet_amount = (max_bet*kelly_home)
        if home_line < 0:
            to_win = bet_amount/((home_line*-1)/100)
        else:
            to_win = bet_amount/((home_line)/100)
        prob = home_pred

    
    elif diff_away > diff_home and diff_away > 0.05:
        bet_amount = (max_bet*kelly_away)
        if away_line < 0:
            to_win = -1*bet_amount/((away_line*-1)/100)
        else:
            to_win = -1*bet_amount/((away_line)/100)
        prob = away_pred

    return bet_amount,to_win,prob

def BNN_kelly(preds,actual,money_lines):
    money_made = 0
    money_risked = 0
    correct = 0
    guessed = 0
    team_bet = []
    amount = []
    gained = []
    probs = []     

    for i in range(len(preds)):
        home_pred = preds[i]
        away_pred = 1 - home_pred
        home_ml = money_lines[i][7]
        away_ml = money_lines[i][10]

        to_bet,to_win,prob = kelly(home_pred,away_pred,home_ml,away_ml)
        probs.append(prob)
        money_risked += to_bet

        curr_gained = 0

        if to_win < 0:
            team_bet.append('Away')
            amount.append(to_bet)
            guessed += 1
            curr_gained = -1*to_bet
            if actual[i] == 1:
                correct += 1
                curr_gained = (-1*to_win)
                #money_made += curr_gained
        elif to_win > 0:
            team_bet.append('Home')
            amount.append(to_bet)
            guessed += 1
            curr_gained = -1*to_bet
            if actual[i] == 0:
                correct += 1
                curr_gained = to_win
                #money_made += curr_gained
        else:
            team_bet.append(0)
            amount.append(0)

        gained.append(curr_gained)

        if curr_gained > 0:
            money_made += curr_gained

    return correct,guessed,team_bet,probs,amount,gained
        

In [79]:
# Get 2022-2023 game data and betting information
features_new = np.genfromtxt('NBA/samps_feats/2022-2023_nba_features_inj.csv',delimiter=',')
samples_new = np.genfromtxt('NBA/samps_feats/2022-2023_nba_samples_inj.csv',delimiter=',')
bet_data = np.genfromtxt('NBA/with_bets/2022-2023_season.csv',delimiter=',')

samples_new_1d = [0 if j[0] == 0 else 1 for j in samples_new]

new_x_tens = torch.FloatTensor(features_new)
new_y_tens = torch.FloatTensor(samples_new)

new_predictive = Predictive(model=model, posterior_samples=mcmc.get_samples(),return_sites=("obs","_RETURN"))
new_y_pred = new_predictive(new_x_tens)['obs'].T.float().mean(axis=1)

correct,guessed,team_bet,probs,amount,gained = BNN_kelly(new_y_pred,samples_new_1d,bet_data[1:])

print(correct)
print(guessed)
print(sum(gained))

48
104
tensor(-465.6457)


In [82]:
# Test on 2023-2024 data
features_new = np.genfromtxt('NBA/samps_feats/2023-2024_nba_features_inj.csv',delimiter=',')
samples_new = np.genfromtxt('NBA/samps_feats/2023-2024_nba_samples_inj.csv',delimiter=',')
bet_data = np.genfromtxt('NBA/with_bets/2023-2024_season.csv',delimiter=',')

samples_new_1d = [0 if j[0] == 0 else 1 for j in samples_new]

new_x_tens = torch.FloatTensor(features_new)
new_y_tens = torch.FloatTensor(samples_new)

new_predictive = Predictive(model=model, posterior_samples=mcmc.get_samples(),return_sites=("obs","_RETURN"))
new_y_pred = new_predictive(new_x_tens)['obs'].T.float().mean(axis=1)

correct,guessed,team_bet,probs,amount,gained = BNN_kelly(new_y_pred,samples_new_1d,bet_data[1:])

print(correct)
print(guessed)
print(sum(gained))

22
56
tensor(-331.8321)


### Initial testing yielded better results than traditional MLPs as seen in ``mlp_betting.ipynb``, explore BNN architecture with more layers
Add a single hidden layer to our existing architecture and increase the number of posterior samples used during MCMC training

In [3]:
class BNN_Multi_Layer(PyroModule):
    def __init__(self, in_dim=1, out_dim=1, first_hid_dim=5, sec_hid_dim=5, prior_scale=1.):
        super().__init__()

        self.activation = nn.ReLU()
        self.layer1 = PyroModule[nn.Linear](in_dim, first_hid_dim)
        self.layer2 = PyroModule[nn.Linear](first_hid_dim, sec_hid_dim)
        self.out = PyroModule[nn.Linear](sec_hid_dim, out_dim)

        # Set layer parameters as random variables
        self.layer1.weight = PyroSample(dist.Normal(0., prior_scale).expand([in_dim, first_hid_dim]).to_event(2))
        self.layer1.bias = PyroSample(dist.Normal(0., prior_scale).expand([first_hid_dim,]).to_event(1))
        self.layer2.weight = PyroSample(dist.Normal(0., prior_scale).expand([first_hid_dim, sec_hid_dim]).to_event(2))
        self.layer2.bias = PyroSample(dist.Normal(0., prior_scale).expand([sec_hid_dim,]).to_event(1)) # output bias term
        self.out.weight = PyroSample(dist.Normal(0., prior_scale).expand([sec_hid_dim, out_dim]).to_event(2))
        self.out.bias = PyroSample(dist.Normal(0., prior_scale).expand([out_dim,]).to_event(1)) # output bias term

    def forward(self, x, y=None):
        z1 = self.activation(x @ self.layer1.weight + self.layer1.bias) # pass thru first layer
        z2 = self.activation(z1 @ self.layer2.weight + self.layer2.bias) # pass thru sec layer
        z3 = self.activation(z2 @ self.out.weight + self.out.bias) # output layer

        #, x.shape[0]
        with pyro.plate("data"):
            y_hat = Softmax(dim=1)(z3)
            obs = pyro.sample("obs", dist.Categorical(y_hat), obs=y)

        return y_hat

Using 2014/2015-2022/2023 NBA seasons to train, make predictions on 2023/2024 games

In [5]:
# load old samples and features
feat_train = np.genfromtxt('../NBA/samps_feats/2015-2023_nba_features_norm_inj.csv',delimiter=',')
samp_train = np.genfromtxt('../NBA/samps_feats/2015-2023_nba_samples_inj.csv',delimiter=',')
samp_train_1d = [0 if j[0] == 0 else 1 for j in samp_train]

feat_test = np.genfromtxt('../NBA/samps_feats/2023-2024_nba_features_inj.csv',delimiter=',') # unnormalized
samp_test = np.genfromtxt('../NBA/samps_feats/2023-2024_nba_samples_inj.csv',delimiter=',') # unnormalized
feat_test_norm = [[float(i)/sum(j) for i in j ]for j in feat_test]
samp_test_1d = [0 if j[0] == 0 else 1 for j in samp_test]

x_train = torch.FloatTensor(feat_train)
x_test = torch.FloatTensor(feat_test_norm)
y_train = torch.FloatTensor(samp_train)
y_test = torch.FloatTensor(samp_test)

In [6]:
new_model = BNN_Multi_Layer(in_dim=16,first_hid_dim=16,sec_hid_dim=16,out_dim=2)

pyro.set_rng_seed(1)

# NUTS = "No-U-Turn Sampler" (https://arxiv.org/abs/1111.4246), gives HMC an adaptive step size
nuts_kernel = NUTS(new_model, jit_compile=True)

mcmc = MCMC(nuts_kernel, num_samples=50)

mcmc.run(x_train, y_train.T)

Sample: 100%|██████████| 100/100 [12:59,  7.80s/it, step size=2.89e-03, acc. prob=0.723]


In [7]:
# get predictions based on posteriors
predictive = Predictive(model=new_model, posterior_samples=mcmc.get_samples(), return_sites=['obs','_RETURN'])

train_preds = predictive(x_train)['obs'].T.float().mean(axis=1)
test_preds = predictive(x_test)['obs'].T.float().mean(axis=1)

adj_train_preds = [0 if p < 0.5 else 1 for p in train_preds]
adj_test_preds = [0 if p < 0.5 else 1 for p in test_preds]
y_train_1d = [0 if j[0] == 0 else 1 for j in y_train] # [0,1] -> [away,home] -> 0 indicates home win, 1 indicates away
y_test_1d = [0 if j[0] == 0 else 1 for j in y_test]

print('---TRAINING SET---')
print('TN, FP, FN, TP')
print(confusion_matrix(y_train_1d,adj_train_preds).ravel())
print(classification_report(y_train_1d,adj_train_preds))
print('---TEST SET---')
print('TN, FP, FN, TP')
print(confusion_matrix(y_test_1d,adj_test_preds).ravel())
print(classification_report(y_test_1d,adj_test_preds))

---TRAINING SET---
TN, FP, FN, TP
[2582 3309 2475 3151]
              precision    recall  f1-score   support

           0       0.51      0.44      0.47      5891
           1       0.49      0.56      0.52      5626

    accuracy                           0.50     11517
   macro avg       0.50      0.50      0.50     11517
weighted avg       0.50      0.50      0.50     11517

---TEST SET---
TN, FP, FN, TP
[272 389 297 356]
              precision    recall  f1-score   support

           0       0.48      0.41      0.44       661
           1       0.48      0.55      0.51       653

    accuracy                           0.48      1314
   macro avg       0.48      0.48      0.48      1314
weighted avg       0.48      0.48      0.48      1314



In [43]:
# load old samples and features
features = np.genfromtxt('../NBA/samps_feats/2015-2023_nba_features_norm_inj.csv',delimiter=',')
samples = np.genfromtxt('../NBA/samps_feats/2015-2023_nba_samples_inj.csv',delimiter=',')
samples_1d = [0 if j[0] == 0 else 1 for j in samples]
feat_train, feat_test, samp_train, samp_test = train_test_split(features, samples, test_size=0.25, random_state=1)
x_train = torch.FloatTensor(feat_train)
x_test = torch.FloatTensor(feat_test)
y_train = torch.FloatTensor(samp_train)
y_test = torch.FloatTensor(samp_test)

In [44]:
new_model = BNN_Multi_Layer(in_dim=16,first_hid_dim=16,sec_hid_dim=16,out_dim=2)

pyro.set_rng_seed(1)

# NUTS = "No-U-Turn Sampler" (https://arxiv.org/abs/1111.4246), gives HMC an adaptive step size
nuts_kernel = NUTS(new_model, jit_compile=True)

mcmc = MCMC(nuts_kernel, num_samples=50)

mcmc.run(x_train, y_train.T)

Warmup:   0%|          | 0/200 [00:00, ?it/s]

Sample: 100%|██████████| 200/200 [27:06,  8.13s/it, step size=3.46e-03, acc. prob=0.594]


In [45]:
# get predictions based on posteriors
predictive = Predictive(model=new_model, posterior_samples=mcmc.get_samples(), return_sites=['obs','_RETURN'])

train_preds = predictive(x_train)['obs'].T.float().mean(axis=1)
test_preds = predictive(x_test)['obs'].T.float().mean(axis=1)

adj_train_preds = [0 if p < 0.5 else 1 for p in train_preds]
adj_test_preds = [0 if p < 0.5 else 1 for p in test_preds]
y_train_1d = [0 if j[0] == 0 else 1 for j in y_train] # [0,1] -> [away,home] -> 0 indicates home win, 1 indicates away
y_test_1d = [0 if j[0] == 0 else 1 for j in y_test]

print('---TRAINING SET---')
print('TN, FP, FN, TP')
print(confusion_matrix(y_train_1d,adj_train_preds).ravel())
print(classification_report(y_train_1d,adj_train_preds))
print('---TEST SET---')
print('TN, FP, FN, TP')
print(confusion_matrix(y_test_1d,adj_test_preds).ravel())
print(classification_report(y_test_1d,adj_test_preds))

---TRAINING SET---
TN, FP, FN, TP
[2041 2359 1925 2312]
              precision    recall  f1-score   support

           0       0.51      0.46      0.49      4400
           1       0.49      0.55      0.52      4237

    accuracy                           0.50      8637
   macro avg       0.50      0.50      0.50      8637
weighted avg       0.50      0.50      0.50      8637

---TEST SET---
TN, FP, FN, TP
[698 793 619 770]
              precision    recall  f1-score   support

           0       0.53      0.47      0.50      1491
           1       0.49      0.55      0.52      1389

    accuracy                           0.51      2880
   macro avg       0.51      0.51      0.51      2880
weighted avg       0.51      0.51      0.51      2880



In [46]:
# Get 2022-2023 game data and betting information
features_new = np.genfromtxt('../NBA/samps_feats/2022-2023_nba_features_inj.csv',delimiter=',')
samples_new = np.genfromtxt('../NBA/samps_feats/2022-2023_nba_samples_inj.csv',delimiter=',')
bet_data = np.genfromtxt('../NBA/with_bets/2022-2023_season.csv',delimiter=',')

samples_new_1d = [0 if j[0] == 0 else 1 for j in samples_new]

new_x_tens = torch.FloatTensor(features_new)
new_y_tens = torch.FloatTensor(samples_new)

new_predictive = Predictive(model=new_model, posterior_samples=mcmc.get_samples(),return_sites=("obs","_RETURN"))
new_y_pred = new_predictive(new_x_tens)['obs'].T.float().mean(axis=1)

correct,guessed,team_bet,probs,amount,gained = BNN_kelly(new_y_pred,samples_new_1d,bet_data[1:])
print(correct)
print(guessed)
print(sum(gained))

35
64
tensor(-19.2364)


In [47]:
# Test on 2023-2024 data
features_new = np.genfromtxt('../NBA/samps_feats/2023-2024_nba_features_inj.csv',delimiter=',')
samples_new = np.genfromtxt('../NBA/samps_feats/2023-2024_nba_samples_inj.csv',delimiter=',')
bet_data = np.genfromtxt('../NBA/with_bets/2023-2024_season.csv',delimiter=',')

samples_new_1d = [0 if j[0] == 0 else 1 for j in samples_new]

new_x_tens = torch.FloatTensor(features_new)
new_y_tens = torch.FloatTensor(samples_new)

new_predictive = Predictive(model=new_model, posterior_samples=mcmc.get_samples(),return_sites=("obs","_RETURN"))
new_y_pred = new_predictive(new_x_tens)['obs'].T.float().mean(axis=1)

correct,guessed,team_bet,probs,amount,gained = BNN_kelly(new_y_pred,samples_new_1d,bet_data[1:])
print(correct)
print(guessed)
print(sum(gained))

16
52
tensor(-316.8422)


### Our new structure yielded better results, however at a significant cost to runtime. Explore the use of Stochastic Variational Inference for training: 
Simple single layer BNN, using SVI with AutoNormal guide

In [96]:
from torch.nn import LogSoftmax, Softmax

class BNN_SVI(PyroModule):
    def __init__(self, in_dim=1, out_dim=1, first_hid_dim=5, prior_scale=1.):
        super().__init__()

        self.activation = nn.ReLU()
        self.layer1 = PyroModule[nn.Linear](in_dim, first_hid_dim)
        self.out = PyroModule[nn.Linear](first_hid_dim, out_dim)

        # Set layer parameters as random variables
        self.layer1.weight = PyroSample(dist.Normal(0., prior_scale).expand([in_dim, first_hid_dim]).to_event(2))
        self.layer1.bias = PyroSample(dist.Normal(0., prior_scale).expand([first_hid_dim,]).to_event(1))
        self.out.weight = PyroSample(dist.Normal(0., prior_scale).expand([first_hid_dim, out_dim]).to_event(2))
        self.out.bias = PyroSample(dist.Normal(0., prior_scale).expand([out_dim,]).to_event(1)) # output bias term

    def forward(self, x, y=None):
        z1 = self.activation(x @ self.layer1.weight + self.layer1.bias) # pass thru first layer
        z2 = self.activation(z1 @ self.out.weight + self.out.bias) # output layer
        y_hat = LogSoftmax(dim=1)(z2) # scale output via softmax for setting categorical priors

        with pyro.plate("data"):
            obs = pyro.sample("obs", dist.Categorical(logits=y_hat).to_event(1), obs=y)

        return y_hat

In [130]:
from sklearn.preprocessing import minmax_scale, maxabs_scale
features = np.genfromtxt('../NBA/samps_feats/2015-2023_nba_features_maxabs.csv',delimiter=',') # unnormalized
samples = np.genfromtxt('../NBA/samps_feats/2015-2023_nba_samples.csv',delimiter=',')
feat_test = maxabs_scale(np.genfromtxt('../NBA/samps_feats/2023-2024_nba_features_inj.csv',delimiter=','))
samp_test = np.genfromtxt('../NBA/samps_feats/2023-2024_nba_samples_inj.csv',delimiter=',')

x_train = torch.FloatTensor(features)
x_test = torch.FloatTensor(feat_test)
y_train = torch.Tensor(samples.T)
y_test = torch.FloatTensor(samp_test)

In [131]:
from pyro.infer.autoguide import AutoNormal
from pyro.optim import ClippedAdam
from pyro.infer import TraceEnum_ELBO
from pyro.optim import Adam

pyro.clear_param_store()
new_model = BNN_SVI(in_dim=16,first_hid_dim=16,out_dim=2)
guide = AutoNormal(new_model)

svi = SVI(new_model, guide, Adam({"lr": 1e-3}), Trace_ELBO())
steps = 2000

for step in range(steps):
    loss = svi.step(x_train,y_train)
    if step % 100 == 0:
        print("[iteration %04d] loss: %.4f" % (step + 1, loss / len(x_train)))

[iteration 0001] loss: 1.5100
[iteration 0101] loss: 1.4795
[iteration 0201] loss: 1.4451
[iteration 0301] loss: 1.4494
[iteration 0401] loss: 1.4377
[iteration 0501] loss: 1.4332
[iteration 0601] loss: 1.4339
[iteration 0701] loss: 1.4340
[iteration 0801] loss: 1.4319
[iteration 0901] loss: 1.4302
[iteration 1001] loss: 1.4300
[iteration 1101] loss: 1.4309
[iteration 1201] loss: 1.4292
[iteration 1301] loss: 1.4282
[iteration 1401] loss: 1.4264
[iteration 1501] loss: 1.4273
[iteration 1601] loss: 1.4241
[iteration 1701] loss: 1.4240
[iteration 1801] loss: 1.4253
[iteration 1901] loss: 1.4234


In [132]:
predictive = Predictive(new_model, guide=guide, num_samples=400, return_sites=["obs","_RETURN"]) # confidence scales inversely w/ num_samples, low num_samp -> high confidence
train_preds = predictive(x_train)['obs'].T.float().mean(axis=2)
test_preds = predictive(x_test)['obs'].T.float().mean(axis=2)

adj_train_preds = [0 if p < 0.5 else 1 for p in train_preds]
adj_test_preds = [0 if p < 0.5 else 1 for p in test_preds]
y_train_1d = [0 if j[0] == 0 else 1 for j in y_train.T] # [0,1] -> [away,home] -> 0 indicates home win, 1 indicates away
y_test_1d = [0 if j[0] == 0 else 1 for j in y_test]

print('---TRAINING SET---')
print('TN, FP, FN, TP')
print(confusion_matrix(y_train_1d,adj_train_preds).ravel())
print(classification_report(y_train_1d,adj_train_preds))
print('---TEST SET---')
print('TN, FP, FN, TP')
print(confusion_matrix(y_test_1d,adj_test_preds).ravel())
print(classification_report(y_test_1d,adj_test_preds))

---TRAINING SET---
TN, FP, FN, TP
[2547 2762 2381 2684]
              precision    recall  f1-score   support

           0       0.52      0.48      0.50      5309
           1       0.49      0.53      0.51      5065

    accuracy                           0.50     10374
   macro avg       0.50      0.50      0.50     10374
weighted avg       0.51      0.50      0.50     10374

---TEST SET---
TN, FP, FN, TP
[312 349 303 350]
              precision    recall  f1-score   support

           0       0.51      0.47      0.49       661
           1       0.50      0.54      0.52       653

    accuracy                           0.50      1314
   macro avg       0.50      0.50      0.50      1314
weighted avg       0.50      0.50      0.50      1314



In [133]:
features_new = maxabs_scale(np.genfromtxt('../NBA/samps_feats/2022-2023_nba_features_inj.csv',delimiter=','))
samples_new = np.genfromtxt('../NBA/samps_feats/2022-2023_nba_samples_inj.csv',delimiter=',')
bet_data = np.genfromtxt('../NBA/with_bets/2022-2023_season.csv',delimiter=',')

samples_new_1d = [0 if j[0] == 0 else 1 for j in samples_new]

new_x_tens = torch.FloatTensor(features_new)
new_y_tens = torch.FloatTensor(samples_new)

new_y_pred = predictive(new_x_tens)['obs'].T.float().mean(axis=2)

correct,guessed,team_bet,probs,amount,gained = BNN_kelly(new_y_pred,samples_new_1d,bet_data[1:])
print(correct)
print(guessed)
print(sum(gained))

# Test on 2023-2024 data
features_new = maxabs_scale(np.genfromtxt('../NBA/samps_feats/2023-2024_nba_features_inj.csv',delimiter=','))
samples_new = np.genfromtxt('../NBA/samps_feats/2023-2024_nba_samples_inj.csv',delimiter=',')
bet_data = np.genfromtxt('../NBA/with_bets/2023-2024_season.csv',delimiter=',')

samples_new_1d = [0 if j[0] == 0 else 1 for j in samples_new]

new_x_tens = torch.FloatTensor(features_new)
new_y_tens = torch.FloatTensor(samples_new)

new_y_pred = predictive(new_x_tens)['obs'].T.float().mean(axis=2)

correct,guessed,team_bet,probs,amount,gained = BNN_kelly(new_y_pred,samples_new_1d,bet_data[1:])
print(correct)
print(guessed)
print(sum(gained))

8
17
tensor([-22.8155])
9
18
tensor([-47.5891])


Simple BNN structure saw significant improvement in runtime, and produces much less confident predictions. Lets try a more complex structure now:
## Multi-Layer BNN w/ SVI

In [134]:
from torch.nn import LogSoftmax

class BNN_Multi_Layer_SVI(PyroModule):
    def __init__(self, in_dim=1, out_dim=1, first_hid_dim=5, sec_hid_dim=5, prior_scale=1.):
        super().__init__()

        self.activation = nn.ReLU()
        self.layer1 = PyroModule[nn.Linear](in_dim, first_hid_dim)
        self.layer2 = PyroModule[nn.Linear](first_hid_dim, sec_hid_dim)
        self.out = PyroModule[nn.Linear](sec_hid_dim, out_dim)

        # Set layer parameters as random variables
        self.layer1.weight = PyroSample(dist.Normal(0., prior_scale).expand([in_dim, first_hid_dim]).to_event(2))
        self.layer1.bias = PyroSample(dist.Normal(0., prior_scale).expand([first_hid_dim,]).to_event(1))
        self.layer2.weight = PyroSample(dist.Normal(0., prior_scale).expand([first_hid_dim, sec_hid_dim]).to_event(2))
        self.layer2.bias = PyroSample(dist.Normal(0., prior_scale).expand([sec_hid_dim,]).to_event(1)) # output bias term
        self.out.weight = PyroSample(dist.Normal(0., prior_scale).expand([sec_hid_dim, out_dim]).to_event(2))
        self.out.bias = PyroSample(dist.Normal(0., prior_scale).expand([out_dim,]).to_event(1)) # output bias term

    def forward(self, x, y=None):
        z1 = self.activation(x @ self.layer1.weight + self.layer1.bias) # pass thru first layer
        z2 = self.activation(z1 @ self.layer2.weight + self.layer2.bias) # pass thru sec layer
        z3 = self.activation(z2 @ self.out.weight + self.out.bias) # output layer

        y_hat = Softmax(dim=1)(z3)

        with pyro.plate("data"):
            obs = pyro.sample("obs", dist.Categorical(y_hat).to_event(1), obs=y)

        return y_hat

In [135]:
from pyro.infer.autoguide import AutoNormal
from pyro.optim import ClippedAdam
from pyro.infer import TraceEnum_ELBO
from pyro.optim import Adam

pyro.clear_param_store()
new_model = BNN_Multi_Layer_SVI(in_dim=16,first_hid_dim=16,sec_hid_dim=16,out_dim=2)
guide = AutoNormal(new_model)

svi = SVI(new_model, guide, Adam({"lr": 1e-3}), Trace_ELBO())
steps = 2000

for step in range(steps):
    loss = svi.step(x_train,y_train)
    if step % 100 == 0:
        print("[iteration %04d] loss: %.4f" % (step + 1, loss / len(x_train)))



[iteration 0001] loss: 1.5503
[iteration 0101] loss: 1.4858
[iteration 0201] loss: 1.4848
[iteration 0301] loss: 1.4958
[iteration 0401] loss: 1.4817
[iteration 0501] loss: 1.4814
[iteration 0601] loss: 1.4788
[iteration 0701] loss: 1.4754
[iteration 0801] loss: 1.4718
[iteration 0901] loss: 1.4735
[iteration 1001] loss: 1.4729
[iteration 1101] loss: 1.4713
[iteration 1201] loss: 1.4709
[iteration 1301] loss: 1.4940
[iteration 1401] loss: 1.4656
[iteration 1501] loss: 1.4654
[iteration 1601] loss: 1.4616
[iteration 1701] loss: 1.4620
[iteration 1801] loss: 1.4586
[iteration 1901] loss: 1.4596


In [136]:
predictive = Predictive(new_model, guide=guide, num_samples=400, return_sites=["obs","_RETURN"]) # confidence scales inversely w/ num_samples, low num_samp -> high confidence
train_preds = predictive(x_train)['obs'].T.float().mean(axis=2)
test_preds = predictive(x_test)['obs'].T.float().mean(axis=2)

adj_train_preds = [0 if p < 0.5 else 1 for p in train_preds]
adj_test_preds = [0 if p < 0.5 else 1 for p in test_preds]
y_train_1d = [0 if j[0] == 0 else 1 for j in y_train.T] # [0,1] -> [away,home] -> 0 indicates home win, 1 indicates away
y_test_1d = [0 if j[0] == 0 else 1 for j in y_test]

print('---TRAINING SET---')
print('TN, FP, FN, TP')
print(confusion_matrix(y_train_1d,adj_train_preds).ravel())
print(classification_report(y_train_1d,adj_train_preds))
print('---TEST SET---')
print('TN, FP, FN, TP')
print(confusion_matrix(y_test_1d,adj_test_preds).ravel())
print(classification_report(y_test_1d,adj_test_preds))

---TRAINING SET---
TN, FP, FN, TP
[2443 2866 2428 2637]
              precision    recall  f1-score   support

           0       0.50      0.46      0.48      5309
           1       0.48      0.52      0.50      5065

    accuracy                           0.49     10374
   macro avg       0.49      0.49      0.49     10374
weighted avg       0.49      0.49      0.49     10374

---TEST SET---
TN, FP, FN, TP
[300 361 313 340]
              precision    recall  f1-score   support

           0       0.49      0.45      0.47       661
           1       0.49      0.52      0.50       653

    accuracy                           0.49      1314
   macro avg       0.49      0.49      0.49      1314
weighted avg       0.49      0.49      0.49      1314



In [137]:
features_new = maxabs_scale(np.genfromtxt('../NBA/samps_feats/2022-2023_nba_features_inj.csv',delimiter=','))
samples_new = np.genfromtxt('../NBA/samps_feats/2022-2023_nba_samples_inj.csv',delimiter=',')
bet_data = np.genfromtxt('../NBA/with_bets/2022-2023_season.csv',delimiter=',')

samples_new_1d = [0 if j[0] == 0 else 1 for j in samples_new]

new_x_tens = torch.FloatTensor(features_new)
new_y_tens = torch.FloatTensor(samples_new)

new_y_pred = predictive(new_x_tens)['obs'].T.float().mean(axis=2)

correct,guessed,team_bet,probs,amount,gained = BNN_kelly(new_y_pred,samples_new_1d,bet_data[1:])
print(correct)
print(guessed)
print(sum(gained))

# Test on 2023-2024 data
features_new = maxabs_scale(np.genfromtxt('../NBA/samps_feats/2023-2024_nba_features_inj.csv',delimiter=','))
samples_new = np.genfromtxt('../NBA/samps_feats/2023-2024_nba_samples_inj.csv',delimiter=',')
bet_data = np.genfromtxt('../NBA/with_bets/2023-2024_season.csv',delimiter=',')

samples_new_1d = [0 if j[0] == 0 else 1 for j in samples_new]

new_x_tens = torch.FloatTensor(features_new)
new_y_tens = torch.FloatTensor(samples_new)

new_y_pred = predictive(new_x_tens)['obs'].T.float().mean(axis=2)

correct,guessed,team_bet,probs,amount,gained = BNN_kelly(new_y_pred,samples_new_1d,bet_data[1:])
print(correct)
print(guessed)
print(sum(gained))

4
14
tensor([-80.8631])
2
7
tensor([-37.8714])


## Trying different normalization technique
Improving quality of data could be useful, we will explore the performance of a simple BNN on unnormalized, minmax norm, maxabs norm

### Unnormalized
First, we will construct a new unnormalized features file from 2014/2015 to 2022/2023

In [4]:
from sklearn.preprocessing import minmax_scale, maxabs_scale
features = []
feat_minmax = []
feat_maxabs = []
samples = []
start = 2014

while start < 2023:
    if start == 2018: # this year is missing and wont populate thru scraper!!
        start += 1
        continue

    curr_feats = np.genfromtxt('../NBA/samps_feats/{start}-{end}_nba_features_inj.csv'.format(start=start,end=start+1),delimiter=',')
    curr_samps = np.genfromtxt('../NBA/samps_feats/{start}-{end}_nba_samples_inj.csv'.format(start=start,end=start+1),delimiter=',')
    feat_minmax.extend(minmax_scale(curr_feats))
    feat_maxabs.extend(maxabs_scale(curr_feats))
    features.extend(curr_feats)
    samples.extend(curr_samps)
    start += 1


np.savetxt('../NBA/samps_feats/2015-2023_nba_features_unnorm.csv', features, delimiter=',')
np.savetxt('../NBA/samps_feats/2015-2023_nba_samples.csv', samples, delimiter=',')
np.savetxt('../NBA/samps_feats/2015-2023_nba_features_minmax.csv', features, delimiter=',')
np.savetxt('../NBA/samps_feats/2015-2023_nba_features_maxabs.csv', features, delimiter=',')

In [5]:
feat_test = np.genfromtxt('../NBA/samps_feats/2023-2024_nba_features_inj.csv',delimiter=',') # unnormalized
samp_test = np.genfromtxt('../NBA/samps_feats/2023-2024_nba_samples_inj.csv',delimiter=',')

x_train = torch.FloatTensor(features)
x_test = torch.FloatTensor(feat_test)
y_train = torch.FloatTensor(samples)
y_test = torch.FloatTensor(samp_test)

C:\Users\andre\AppData\Local\Temp\ipykernel_15840\2636246682.py:4: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:277.)
  x_train = torch.FloatTensor(features)


In [6]:
from pyro.infer import MCMC, NUTS

x_train = torch.FloatTensor(feat_maxabs)
x_test = torch.FloatTensor(maxabs_scale(feat_test))
y_train = torch.FloatTensor(samples)
y_test = torch.FloatTensor(samp_test)

model = BNN(in_dim=16,hid_dim=16,out_dim=2)

pyro.set_rng_seed(1)

# Define Hamiltonian Monte Carlo (HMC) kernel
# NUTS = "No-U-Turn Sampler" (https://arxiv.org/abs/1111.4246), gives HMC an adaptive step size
nuts_kernel = NUTS(model, jit_compile=False)  # jit_compile=True is faster but requires PyTorch 1.6+

# Define MCMC sampler, get 50 posterior samples
mcmc = MCMC(nuts_kernel, num_samples=50)

mcmc.run(x_train, y_train.T)

Sample: 100%|██████████| 100/100 [01:35,  1.05it/s, step size=4.07e-02, acc. prob=0.788]


In [7]:
# get predictions based on posteriors
predictive = Predictive(model=model, posterior_samples=mcmc.get_samples(), return_sites=['obs','_RETURN'])

train_preds = predictive(x_train)['obs'].T.float().mean(axis=1)
test_preds = predictive(x_test)['obs'].T.float().mean(axis=1)

adj_train_preds = [0 if p < 0.5 else 1 for p in train_preds]
adj_test_preds = [0 if p < 0.5 else 1 for p in test_preds]
y_train_1d = [0 if j[0] == 0 else 1 for j in y_train] # [0,1] -> [away,home] -> 0 indicates home win, 1 indicates away
y_test_1d = [0 if j[0] == 0 else 1 for j in y_test]

print('---TRAINING SET---')
print('TN, FP, FN, TP')
print(confusion_matrix(y_train_1d,adj_train_preds).ravel())
print(classification_report(y_train_1d,adj_train_preds))
print('---TEST SET---')
print('TN, FP, FN, TP')
print(confusion_matrix(y_test_1d,adj_test_preds).ravel())
print(classification_report(y_test_1d,adj_test_preds))

---TRAINING SET---
TN, FP, FN, TP
[2384 2925 2191 2874]
              precision    recall  f1-score   support

           0       0.52      0.45      0.48      5309
           1       0.50      0.57      0.53      5065

    accuracy                           0.51     10374
   macro avg       0.51      0.51      0.51     10374
weighted avg       0.51      0.51      0.51     10374

---TEST SET---
TN, FP, FN, TP
[296 365 267 386]
              precision    recall  f1-score   support

           0       0.53      0.45      0.48       661
           1       0.51      0.59      0.55       653

    accuracy                           0.52      1314
   macro avg       0.52      0.52      0.52      1314
weighted avg       0.52      0.52      0.52      1314



In [8]:
# Get 2022-2023 game data and betting information
features_new = maxabs_scale(np.genfromtxt('../NBA/samps_feats/2022-2023_nba_features_inj.csv',delimiter=','))
samples_new = np.genfromtxt('../NBA/samps_feats/2022-2023_nba_samples_inj.csv',delimiter=',')
bet_data = np.genfromtxt('../NBA/with_bets/2022-2023_season.csv',delimiter=',')

samples_new_1d = [0 if j[0] == 0 else 1 for j in samples_new]

new_x_tens = torch.FloatTensor(features_new)
new_y_tens = torch.FloatTensor(samples_new)

new_predictive = Predictive(model=model, posterior_samples=mcmc.get_samples(),return_sites=("obs","_RETURN"))
new_y_pred = new_predictive(new_x_tens)['obs'].T.float().mean(axis=1)

correct,guessed,team_bet,probs,amount,gained = BNN_kelly(new_y_pred,samples_new_1d,bet_data[1:])
print(correct)
print(guessed)
print(sum(gained))

# Test on 2023-2024 data
features_new = maxabs_scale(np.genfromtxt('../NBA/samps_feats/2023-2024_nba_features_inj.csv',delimiter=','))
samples_new = np.genfromtxt('../NBA/samps_feats/2023-2024_nba_samples_inj.csv',delimiter=',')
bet_data = np.genfromtxt('../NBA/with_bets/2023-2024_season.csv',delimiter=',')

samples_new_1d = [0 if j[0] == 0 else 1 for j in samples_new]

new_x_tens = torch.FloatTensor(features_new)
new_y_tens = torch.FloatTensor(samples_new)

new_predictive = Predictive(model=model, posterior_samples=mcmc.get_samples(),return_sites=("obs","_RETURN"))
new_y_pred = new_predictive(new_x_tens)['obs'].T.float().mean(axis=1)

correct,guessed,team_bet,probs,amount,gained = BNN_kelly(new_y_pred,samples_new_1d,bet_data[1:])
print(correct)
print(guessed)
print(sum(gained))

40
106
tensor(-746.6345)
49
118
tensor(-704.2104)
